In [1]:
import re
import sys
import os
from pyspark import SparkConf, SparkContext

In [25]:
sc.stop()

In [26]:
conf = SparkConf()
sc = SparkContext(conf=conf)
lines = sc.textFile("hw1/test.txt")

In [113]:
lines = lines.filter(lambda l: len(l.split())>1)
friend_list = lines.map(lambda l:(l.split()[0], l.split()[1].split(',')))
friend_pairs = friend_list.flatMapValues(lambda l: l)

print(friend_pairs.collect())

[('1', '6'), ('1', '11'), ('1', '18'), ('2', '6'), ('2', '9'), ('2', '12'), ('2', '16'), ('2', '21'), ('2', '24'), ('2', '29'), ('3', '0'), ('3', '5'), ('3', '9'), ('3', '10'), ('3', '25'), ('3', '28'), ('4', '5'), ('4', '6'), ('4', '8'), ('4', '14'), ('4', '16'), ('4', '22'), ('4', '25'), ('4', '27'), ('5', '3'), ('5', '4'), ('5', '14'), ('5', '29'), ('6', '1'), ('6', '2'), ('6', '4'), ('6', '7'), ('6', '8'), ('6', '14'), ('6', '16'), ('6', '21'), ('6', '28'), ('6', '29'), ('7', '6'), ('7', '11'), ('7', '13'), ('7', '16'), ('8', '0'), ('8', '4'), ('8', '6'), ('8', '9'), ('8', '12'), ('8', '14'), ('8', '22'), ('8', '29'), ('9', '2'), ('9', '3'), ('9', '8'), ('9', '13'), ('9', '17'), ('9', '26'), ('10', '3'), ('10', '11'), ('10', '20'), ('11', '1'), ('11', '7'), ('11', '10'), ('11', '22'), ('11', '24'), ('11', '27'), ('11', '29'), ('12', '2'), ('12', '8'), ('12', '16'), ('12', '20'), ('12', '26'), ('13', '7'), ('13', '9'), ('13', '15'), ('13', '16'), ('13', '19'), ('13', '24'), ('14', '

In [119]:
def mutual(pair):
    pairs = []
    for x in pair[1]:
        for y in pair[1]:
            if(x != y):
                pairs.append((x,y))
    return pairs

def rec_list(pair):
    rec = str(pair[0]) + "    "
    rec_list = []
    last_val = -1
    for (_id, value) in pair[1]:
        if(value == last_val):
            rec_list.append((_id,value))
        if(len(rec_list) >= 10):
            continue
        rec_list.append((_id, value))
        if(len(rec_list) == 10):
            last_val = value
    first = False
    for (_id, value) in rec_list:
        if(first):
            rec += ","
        first = True
        rec += str(_id)
    return rec
    
mutual_pairs = friend_list.flatMap(mutual).subtract(friend_pairs)
mutual_list = mutual_pairs.map(lambda w: ((int(w[0]), int(w[1])), 1)).reduceByKey(lambda n1, n2: n1 + n2)
grouped = mutual_list.map(lambda w: (w[0][0], (w[0][1], w[1]))).sortBy(lambda w: (w[0], -w[1][1])).groupByKey().mapValues(list).sortBy(lambda w: w[0])
res = grouped.map(rec_list)
for item in res.collect():
    print(item)


0    19,14,6,5,9,29,25,4,8,11
1    29,27,7,24,28,21,10,26,2,14,22,4,16,8
2    8,27,26,13,4,20,11,7,15,19,14,25
3    4,29,11,8,20,16,17,13,6,18,14,2,26
4    29,2,21,17,0,12,20,3,7,23,11
5    25,6,8,27,0,9,17,2,10,22,11,15,19,28,16,20
6    12,11,27,22,25,5,0,20,18,9,13
7    27,4,24,1,21,29,22,2,15,23,19,8,28,12,25,9,14,10
8    2,16,5,17,20,26,25,27,11,19
9    12,21,29,14,22,6,19,16,24,0
10    22,0,24,12,28,7,27,14,9,1,29,17,21,25,5
11    19,16,6,18,15,20,4,0,8,13,2
12    21,22,6,9,14,29,4,18,10,17,25,13,0,24,19,7,23,27
13    2,29,26,6,22,11,27,25,17,21,18,3,23,4,12,8,0
14    22,16,12,0,27,2,21,9,25,28,11,19,3,7,15,10,1
15    11,27,2,7,8,16,25,9,5,18,6,22,14,26
16    29,8,24,20,11,14,26,9,19,18,1,17,5,0,28,15,3
17    8,4,13,5,29,21,26,10,2,6,11,19,3,12,16,0
18    11,19,6,12,4,16,15,23,3,2,9,21,29,13
19    11,0,27,16,8,9,2,18,7,4,20,12,5,17,21,25,14,6
20    8,16,6,26,2,4,11,29,9,5,0,19,3
21    12,9,29,14,22,7,4,13,1,17,25,10,18,27,19,23,24,28,8
22    14,12,27,6,29,24,7,10,13,9,21,25
23    